#Import Libraries

In [ ]:
!pip install transformers
# !pip install flair

In [3]:
import tensorflow.keras as keras
import pandas as pd

import numpy as np
from numpy import zeros
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding, Bidirectional, LSTM, GRU, Flatten, LayerNormalization, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

from nltk.tokenize import TweetTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import DistilBertTokenizer, TFDistilBertModel
# from transformers import BertTokenizer, TFBertModel, TFGPT2Model, GPT2Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import re

In [ ]:
print(help(BertTokenizer))

###Not Used at this time

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModel.from_pretrained("bert-base-uncased")
# from pandas_profiling import ProfileReport


# dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# try: # this is only working on the 2nd try in colab :)
#   from transformers import DistilBertTokenizer, TFDistilBertModel
# except Exception as err: # so we catch the error and import it again
#   from transformers import DistilBertTokenizer, TFDistilBertModel

# import flair
# from flair.embeddings import WordEmbeddings
# from flair.embeddings import ELMoEmbeddings
# from flair.embeddings import TransformerWordEmbeddings
# from flair.data import Sentence

##Explore Raw Data

In [ ]:
#load the data
raw_data_df = pd.read_csv('https://query.data.world/s/twuhmzuhvitwqqcjh5picrq3qykr4r')

In [ ]:
raw_data_df['class'].unique()

array([2, 1, 0])

In [ ]:
class_weight = compute_class_weight('balanced', np.arange(3), raw_data_df['class'])

In [ ]:
class_weight

array([5.77692308, 0.43048463, 1.98438626])

In [ ]:
class_wedight_dict = dict((w, i) for w, i in enumerate(class_weight))

In [ ]:
class_wedight_dict

{0: 5.776923076923077, 1: 0.43048462741010945, 2: 1.9843862599087196}

In [ ]:
y_test = prepare_target(raw_data_df['class'])

In [ ]:
sum(y_test[:,1])

19190.0

In [ ]:
raw_data_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
test = clean_data(raw_df['tweet'][0])

"! <repeat> rt  <allcaps> <user> : as a woman you shouldn't complain about cleaning up your house . &amp ; as a man you should always take the trash out . <repeat>"

In [ ]:
tweeter = TweetTokenizer(strip_handles=True)
tweet_tokenized = tweet_tokenizer(raw_data_df['tweet'][0:5])

In [ ]:
print(tweet_tokenized, '\n', raw_data_df['tweet'][0:5])

[['!', '!', '!', 'RT', ':', 'As', 'a', 'woman', 'you', "shouldn't", 'complain', 'about', 'cleaning', 'up', 'your', 'house', '.', '&', 'as', 'a', 'man', 'you', 'should', 'always', 'take', 'the', 'trash', 'out', '...'], ['!', '!', '!', 'RT', ':', 'boy', 'dats', 'cold', '...', 'tyga', 'dwn', 'bad', 'for', 'cuffin', 'dat', 'hoe', 'in', 'the', '1st', 'place', '!', '!'], ['!', '!', '!', 'RT', 'Dawg', '!', '!', '!', 'RT', ':', 'You', 'ever', 'fuck', 'a', 'bitch', 'and', 'she', 'start', 'to', 'cry', '?', 'You', 'be', 'confused', 'as', 'shit'], ['!', '!', '!', 'RT', ':', 'she', 'look', 'like', 'a', 'tranny'], ['!', '!', '!', 'RT', ':', 'The', 'shit', 'you', 'hear', 'about', 'me', 'might', 'be', 'true', 'or', 'it', 'might', 'be', 'faker', 'than', 'the', 'bitch', 'who', 'told', 'it', 'to', 'ya', '\ue011']] 
 0    !!! RT @mayasolovely: As a woman you shouldn't...
1    !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2    !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3    !!!!!!!!! RT @C_G_And

In [ ]:
flat_list = [item for sublist in tweet_tokenized for item in sublist]
a = np.unique(np.array(flat_list))

In [ ]:
a.shape[0]

68

In [ ]:
kt = Tokenizer()
kt.fit_on_texts(raw_data_df['tweet'][0:5])
word_index = kt.word_index

In [ ]:
word_index.items()

dict_items([('rt', 1), ('you', 2), ('a', 3), ('the', 4), ('as', 5), ('be', 6), ('about', 7), ('bitch', 8), ('she', 9), ('to', 10), ('shit', 11), ('might', 12), ('it', 13), ('mayasolovely', 14), ('woman', 15), ("shouldn't", 16), ('complain', 17), ('cleaning', 18), ('up', 19), ('your', 20), ('house', 21), ('amp', 22), ('man', 23), ('should', 24), ('always', 25), ('take', 26), ('trash', 27), ('out', 28), ('mleew17', 29), ('boy', 30), ('dats', 31), ('cold', 32), ('tyga', 33), ('dwn', 34), ('bad', 35), ('for', 36), ('cuffin', 37), ('dat', 38), ('hoe', 39), ('in', 40), ('1st', 41), ('place', 42), ('urkindofbrand', 43), ('dawg', 44), ('80sbaby4life', 45), ('ever', 46), ('fuck', 47), ('and', 48), ('start', 49), ('cry', 50), ('confused', 51), ('c', 52), ('g', 53), ('anderson', 54), ('viva', 55), ('based', 56), ('look', 57), ('like', 58), ('tranny', 59), ('shenikaroberts', 60), ('hear', 61), ('me', 62), ('true', 63), ('or', 64), ('faker', 65), ('than', 66), ('who', 67), ('told', 68), ('ya', 69),

In [ ]:
tweet_tokenized = [tweeter.tokenize(l) for l in raw_data_df['tweet']]

In [ ]:
tdf = pd.DataFrame(tweet_tokenized)

In [ ]:
np.unique(tdf.values)

In [ ]:
tv, wi = keras_tokenizer(raw_data_df['tweet'])

In [ ]:
print(tv[0], "\n", raw_data_df['tweet'][0],"\n", wi)


In [ ]:
tt = tweet_tokenizer(raw_data_df['tweet'])

In [ ]:
raw_data_df['class']

0        2
1        1
2        1
3        1
4        1
        ..
24778    1
24779    2
24780    1
24781    1
24782    2
Name: class, Length: 24783, dtype: int64

In [ ]:
target_test = prepare_target(raw_data_df['class'])

In [ ]:
target_test.T

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.]], dtype=float32)

#Data Preparation

In [4]:
#load data
def load_data():
  raw_data_df = pd.read_csv('https://query.data.world/s/twuhmzuhvitwqqcjh5picrq3qykr4r')
  return raw_data_df

###Preprocessing 

In [4]:
###############################################################################################################
#clean data copied from https://www.kaggle.com/amackcrane/python-version-of-glove-twitter-preprocess-script
FLAGS = re.MULTILINE | re.DOTALL
def hashtag(text):
  text = text.group()
  hashtag_body = text[1:]
  if hashtag_body.isupper():
      result = "<hashtag> {} <allcaps>".format(hashtag_body.lower())
  else:
      result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
  return result

def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps> " # amackcrane added trailing space
    # function so code less repetitive

def clean_data(text):
  eyes = r"[8:=;]"
  nose = r"['`\-]?"
  def re_sub(pattern, repl):
      return re.sub(pattern, repl, text, flags=FLAGS)

  text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
  text = re_sub(r"@\w+", "<user>")
  text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
  text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
  text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
  text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
  text = re_sub(r"/"," / ")
  text = re_sub(r"<3","<heart>")
  text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
  text = re_sub(r"#\w+", hashtag)  # amackcrane edit
  text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
  text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")
    

  ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
  # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
  #text = re_sub(r"([A-Z]){2,}", allcaps)  # moved below -amackcrane

  # amackcrane additions
  text = re_sub(r"([a-zA-Z<>()])([?!.:;,])", r"\1 \2")
  text = re_sub(r"\(([a-zA-Z<>]+)\)", r"( \1 )")
  text = re_sub(r"  ", r" ")
  text = re_sub(r" ([A-Z]){2,} ", allcaps)
    
  return text.lower()

def preprocessing_tweet(tweet):
  for t in tweet:
    t = clean_data(t)
  return tweet
###########################################################################################################

In [12]:
#split train val test
def split_data(cleantweet):
  trainX, tempX = train_test_split(cleantweet, test_size=0.4, random_state=42)
  valX, testX = train_test_split(tempX, test_size=0.5, random_state=42)
  return trainX, valX, testX

#extract tweet and y
def extract_tweet_and_y(raw_data_df):
  tweet, target = raw_data_df['tweet'], raw_data_df['class']
  return tweet, target

#tokenize and vectorize input using keras tokenizer
def keras_tokenizer(tweet_train, tweet_val, tweet_test, maxnumwords):
  # maxnumwords = 2000
  kt = Tokenizer()
  kt.fit_on_texts(tweet_train)
  word_index = kt.word_index
  vocab_size = len(word_index) + 1

  train_vectors = kt.texts_to_sequences(tweet_train) #Converting text to a vector of word indexes
  val_vectors = kt.texts_to_sequences(tweet_val) #Converting text to a vector of word indexes
  test_vectors = kt.texts_to_sequences(tweet_test) #Converting text to a vector of word indexes
  
  train_padded = pad_sequences(train_vectors, maxlen=maxnumwords, padding='post')
  val_padded = pad_sequences(val_vectors, maxlen=maxnumwords, padding='post')
  test_padded = pad_sequences(test_vectors, maxlen=maxnumwords, padding='post')

  return  train_padded, val_padded, test_padded, vocab_size, word_index

#GloVe embeddings using Glove twiter 100D
def GloveTwitterEmbedding(vocab_size, word_index):
  
  #Glove Twitter 100d
  embedding_path = "/content/drive/MyDrive/Colab Notebooks/GloVe Twitter 27B/glove.twitter.27B.100d.txt"
  # max_features = 30000
  # get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
  # embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))
  embedding_index = dict(
      (o.strip().split(" ")[0], np.array(o.strip().split(" ")[1:], dtype="float32")
      ) for o in open(embedding_path)
      )
  # embedding matrix
  embedding_matrix = zeros((vocab_size, 100))
  # for word, i in enumerate(tweet_tokenized):
  for t , i in enumerate(word_index.items()):
    embedding_vector = embedding_index.get(t)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector
  
  return embedding_matrix

# HuggingFace Transformers AutoTokenizer
def hf_auto_tokenizer(tweet, maxnumwords):
  
  autoTokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
  tweet_tokenized = autoTokenizer(tweet.tolist(), padding = 'max_length',
                                  truncation = True, max_length = maxnumwords, return_tensors='tf')
  input_ids = tweet_tokenized['input_ids']
  att_mask = tweet_tokenized['attention_mask']
  return input_ids, att_mask

#HuggingFace GPT2 Tokenizer
def hf_GPT2_tokenizer(tweet, maxnumwords):

  gpt2Tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  tweet_tokenized = gpt2Tokenizer(tweet.tolist(), padding = 'max_length', truncation = True, max_length = maxnumwords, return_tensors='tf')
  input_ids = tweet_tokenized['input_ids']
  att_mask = tweet_tokenized['attention_mask']
  return input_ids, att_mask

#one hot encode y
def prepare_target(raw_y):
  class_weight = compute_class_weight('balanced', np.arange(3), raw_y)
  class_weight_dict = dict((c,w) for c, w in enumerate(class_weight))
  target = to_categorical(raw_y)
  return np.array(target), class_weight_dict


###Not Used

In [ ]:
# #GloVe embeddings DO NOT USE
# def GloveEmbeddingmodel2(tweet):
#   glove_embedding = WordEmbeddings('glove')
#   tweet_glove_embedding=[]
#   for t in tweet:
#     temp_embedding=[]
#     sentence = Sentence(t)
#     glove_embedding.embed(sentence)
#     for token in sentence:
#       temp_embedding.append(token.embedding)
#     tweet_glove_embedding.append(temp_embedding)
#   return tweet_glove_embedding


# #tokenizers
# def tweet_tokenizer(tweet):
#   tt = TweetTokenizer(strip_handles=True)
#   tweet_tokenized = [tt.tokenize(l) for l in tweet]
#   return tweet_tokenized

#   tweet_tokenized = tweet_tokenizer(tweet)
#   tweet_tokenized_flatten = [item for sublist in tweet_tokenized for item in sublist]
#   tweet_tokenized_unique = np.unique(np.array(tweet_tokenized_flatten))
#   vocab_size = len(tweet_tokenized_unique)

#doc2vec embeddings using traing data only
# def build_d2vmodel(tweet_tokenized):
#   #prepare training data in doc2vec format:
#   train_doc2vec = [TaggedDocument((f), tags=[str(i)]) for i, f in enumerate(tweet_tokenized)]
#   #Train a doc2vec model to learn tweet representations. Use only training data!!
#   D2V_model = Doc2Vec(vector_size=50, alpha=0.025, min_count=5, dm =1, epochs=100)
#   D2V_model.build_vocab(train_doc2vec)
#   D2V_model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)
#   D2V_model.save("d2v.model")

# #vectorize tweet using Doc2Vec
# def encod_tweet_d2v(tweet_tokenized):
#   #Infer the feature representation for training and test data using the trained model
#   model= Doc2Vec.load("d2v.model")
#   #infer in multiple steps to get a stable representation. 
#   tweet_vectors =  [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in tweet_tokenized]

# #word 2 vec embeddings, 
# def build_w2vmodel(tweet_tokenized):
#   pass

# #one hot encode
# def one_hot_encode(text):
#   vocab_size = 100
#   max_length = 4
#   text_one_hot = [one_hot(t, vocab_size) for t in text]
#   return text_one_hot

#Build Albert's Model & Transfer Learning Model

###Albert Model: BiDirectional LSTM

In [4]:
def albert_model(param={}):
  #Bi Directional LSTM
  max_seq_len = param['Max Length']
  inputs = Input(shape = (max_seq_len,), dtype='int64', name='inputs')

  vocab_size = param['Vocab Size']

  embedding_trainable = True
  e = Embedding(vocab_size, 100, embeddings_initializer ='uniform', 
                input_length=max_seq_len, trainable = embedding_trainable)
  
  embedding_matrix = param['Embedding Matrix']
  if embedding_matrix is not None:
    embedding_trainable = False
    e = Embedding(vocab_size, 100, embeddings_initializer ='uniform', input_length=max_seq_len,
                      weights = [embedding_matrix], trainable = embedding_trainable)

  model = Sequential()
  model.add(inputs)
  model.add(e)
  model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=param['dropout']), merge_mode='concat'))
  model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=param['dropout']),merge_mode='concat'))
  model.add(Flatten())
  model.add(LayerNormalization())
  model.add(Dense(param['fist_layer'], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
  model.add(Dropout(param['dropout']))
  model.add(LayerNormalization())
  model.add(Dense(param['secon_layer'], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
  model.add(Dropout(param['dropout']))
  model.add(Dense(3, activation='softmax'))

  model.summary()
  return model

# input = Input(shape=(max_len,))
# model = Embedding(vocab_size,100,weights=[embedding_matrix],input_length=max_len)(input)
# model =  Bidirectional (LSTM (100,return_sequences=True,dropout=0.50),merge_mode='concat')(model)
# model = TimeDistributed(Dense(100,activation='relu'))(model)
# model = Flatten()(model)
# model = Dense(100,activation='relu')(model)
# output = Dense(3,activation='softmax')(model)
# model = Model(input,output)
# model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

###Transfer Learning Models, Bert, DistilBert & GPT2



In [13]:
def tl_disbert_model(param={}):
  
  trainable = param['Trainable']
  max_seq_len = param['Max_length']
  inputs = Input(shape= (max_seq_len,), dtype ='int64', name='inputs')
  masks = Input(shape = (max_seq_len,), dtype='int64', name='masks')

  disBert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
  disBert_model.trainable = param['Trainable']

  disBert_output = disBert_model(inputs, attention_mask = masks)
  disBert_last_hidden = disBert_output.last_hidden_state
  disBert_CLS_output =  disBert_last_hidden [:,0,:]
  x = Flatten()(disBert_CLS_output)
  x = Dense(param['first_layer'], activation='relu')(x)
  x = Dropout(param['dropout'])(x)
  x = Dense(param['second_layer'], activation='relu')(x)
  x = Dropout(param['dropout'])(x)

  probs = Dense(3, activation='softmax')(x)

  model = keras.Model(inputs = [inputs, masks], outputs=probs)
  model.summary()

  return model

def tl_bert_model(param={}):

  
  max_seq_len = param['Max_length']
  inputs = Input(shape= (max_seq_len,), dtype ='int64', name='inputs')
  masks = Input(shape = (max_seq_len,), dtype='int64', name='masks')

  Bert_model = TFBertModel.from_pretrained('bert-base-uncased')
  Bert_model.trainable = param['Trainable']

  Bert_output = Bert_model_model(inputs, attention_mask = masks)
  Bert_last_hidden = Bert_output.last_hidden_state
  Bert_CLS_output =  Bert_last_hidden [:,0,:]
  x = Dense(param['first_layer'], activation='relu', kernel_regularizer=regularizers.l2(0.01) )(Bert_CLS_output)
  x = Dropout(param['dropout'])(x)
  x = Dense(param['second_layer'], activation='relu')(x)
  x = Dropout(param['dropout'])(x)

  probs = Dense(3, activation='softmax')(x)

  model = keras.Model(inputs = [inputs, masks], outputs=probs)
  model.summary()
  return model


In [14]:
def tl_gpt2_model(param={}):
  
  trainable = param['Trainable']
  max_seq_len = param['Max_length']
  inputs = Input(shape= (max_seq_len,), dtype ='int64', name='inputs')
  masks = Input(shape = (max_seq_len,), dtype='int64', name='masks')

  gpt2_model = TFGPT2Model.from_pretrained('gpt2')
  gpt2_model.trainable = param['Trainable']

  gpt2_output = gpt2_model(inputs, attention_mask = masks)
  gpt2_last_hidden = gpt2_output.last_hidden_state
  gpt2_CLS_output =  gpt2_last_hidden[:,0,:]
  x = Flatten()(gpt2_CLS_output)
  x = Dense(param['first_layer'], activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
  x = Dropout(param['dropout'])(x)
  x = Dense(param['second_layer'], activation='relu')(x)
  x = Dropout(param['dropout'])(x)

  probs = Dense(3, activation='softmax')(x)

  model = keras.Model(inputs = [inputs, masks], outputs=probs)
  model.summary()
  return model

###Assemble_model

In [ ]:
def tl_assemble_model():
  pass


###Train, Compile, Evaluate model method

In [9]:
def train_model(model, tweet_train, y_train, tweet_val, y_val, batch_size, num_epochs, class_weight):
  
  es = keras.callbacks.EarlyStopping(monitor="val_loss", mode='min', verbose=1, patience=3)
  history = model.fit(
            tweet_train, y_train,
            batch_size=batch_size,
            epochs=num_epochs,
            verbose=1,
            validation_data=(tweet_val, y_val),
            class_weight = class_weight,
            callbacks=[es])
  return model, history

def compile_model(model):
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics=['accuracy', 
                         keras.metrics.AUC(curve="ROC", multi_label=True), 
                         keras.metrics.AUC(curve="PR", multi_label=True), 
                         keras.metrics.Precision(),
                         keras.metrics.Recall()])
  return model
  
def evaluate_model(model, model_inputs_and_masks_test, y_test):
    # TODO: evaluate the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION
    # HINT: for pr_auc: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html 

    probs = model(model_inputs_and_masks_test, training=False)
    print(probs)
    print(y_test)

    eval_dict = {
        "Hate": {
            "pr_auc": average_precision_score(y_test[:, 0], probs[:, 0]), "pr_auc_random_guess": sum(y_test[:, 0])/(1.0*y_test.shape[0]), 
            "roc_auc": roc_auc_score(y_test[:, 0], probs[:, 0]), "roc_auc_random_guess": 0.5, 
            "precision": precision_score(y_test[:, 0], probs[:, 0] > 0.2),
            "recall": recall_score(y_test[:, 0], probs[:, 0] > 0.2)
        }, 
        "Offensive": {
            "pr_auc": average_precision_score(y_test[:, 1], probs[:, 1]), "pr_auc_random_guess": sum(y_test[:, 1])/(1.0*y_test.shape[0]), 
            "roc_auc": roc_auc_score(y_test[:, 1], probs[:, 1]), "roc_auc_random_guess": 0.5, 
            "precision": precision_score(y_test[:, 1], probs[:, 1] > 0.2),
            "recall": recall_score(y_test[:, 1], probs[:, 1] > 0.2)
        }, 
        "Neither": {
            "pr_auc": average_precision_score(y_test[:, 2], probs[:, 2]), "pr_auc_random_guess": sum(y_test[:, 2])/(1.0*y_test.shape[0]), 
            "roc_auc": roc_auc_score(y_test[:, 2], probs[:,2]), "roc_auc_random_guess": 0.5, 
            "precision": precision_score(y_test[:, 2], probs[:, 2] > 0.2),
            "recall": recall_score(y_test[:, 2], probs[:, 2] > 0.2)
        }
    }
    return eval_dict

#Execution

###Load, Clean & Split Data

In [15]:
raw_df = load_data()

clean_df = raw_df
# clean_df['tweet'] = preprocessing_tweet(clean_df['tweet'])

train_df, val_df, test_df = split_data(clean_df)

X_train, y_train = extract_tweet_and_y(train_df)
X_val, y_val = extract_tweet_and_y(val_df)
X_test, y_test = extract_tweet_and_y(test_df)


y_raw, class_weight_raw = prepare_target(clean_df['class'])
y_train, class_weight_train = prepare_target(y_train)
y_val, class_weight_val = prepare_target(y_val)
y_test, class_weight_test = prepare_target(y_test)

###Tokenize and vectorize input

In [46]:
#Albert Model Tokenizer
X_train_albert, X_val_albert, X_test_albert, vocab_size, word_index = keras_tokenizer(X_train,X_val,X_test, maxnumwords=100)

#Use GloVe Embedding
embedding_matrix = GloveTwitterEmbedding(vocab_size, word_index)

In [17]:
#Transformers Tokenizer
X_train_bert, Attention_mask_train = hf_auto_tokenizer(X_train, maxnumwords=60)
X_val_bert, Attention_mask_val = hf_auto_tokenizer(X_val, maxnumwords=60)
X_test_bert, Attention_mask_test = hf_auto_tokenizer(X_test, maxnumwords =60)


train_bert_inputs_and_masks = {
    'inputs' : X_train_bert,
    'masks' :  Attention_mask_train
}

val_bert_inputs_and_masks = {
    'inputs' : X_val_bert,
    'masks' : Attention_mask_val
}

test_bert_inputs_and_masks = {
    'inputs' : X_test_bert,
    'masks' : Attention_mask_test
}

In [ ]:
#GPT2 Tokenizer
X_train_gpt, Att_mask_train_gpt = hf_GPT2_tokenizer(X_train, maxnumwords=100)
X_val_gpt, Att_mask_val_gpt = hf_GPT2_tokenizer(X_val, maxnumwords=100)
X_test_gpt, Att_mask_test_gpt = hf_GPT2_tokenizer(X_test, maxnumwords =100)


train_gpt_inputs_and_masks = {
    'inputs' : X_train_gpt,
    'masks' :  Att_mask_train_gpt
}

val_gpt_inputs_and_masks = {
    'inputs' : X_val_gpt,
    'masks' : Att_mask_val_gpt
}

test_gpt_inputs_and_masks = {
    'inputs' : X_test_gpt,
    'masks' : Att_mask_test_gpt
}

###Train Models

In [ ]:
#Albert Model
Albertmodel = albert_model(param={'Max_length': 100,
                                  'Vocab Size': vocab_size,
                                  'Embedding Matrix': embedding_matrix,
                                  'dropout':0.50,
                                  'first_layer' : 128,
                                  'second_layer' : 64,
                                  })
Albertmodel = compile_model(Albertmodel)

Albertmodel, history_Albert = train_model(Albertmodel, X_train_albert, y_train, X_val_albert, y_val, batch_size=128, num_epochs=50, class_weight=class_weight_train)

In [18]:
DistilBertmodel = tl_disbert_model(param={'Max_length': 60, 
                                             'Trainable' : False,
                                             'first_layer' : 128,
                                             'second_layer' : 64,
                                             'dropout' :0.50
                                             })
DistilBertmodel = compile_model(DistilBertmodel)

DistilBertmodel, history_disbert =  train_model(DistilBertmodel, train_bert_inputs_and_masks, y_train, val_bert_inputs_and_masks, y_val, batch_size=128, num_epochs=1, class_weight=class_weight_train)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 60)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 60)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti TFBaseModelOutput(la 66362880    inputs[0][0]                     
                                                                 masks[0][0]                      
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 768)          0           tf_distil_bert_model_1[0][0

In [ ]:
Bertmodel = tl_bert_model(param={'Max Length': 100,
                                  'Trainable' : False,
                                  'first_layer' : 128,
                                  'second_layer' : 64,
                                  'dropout' :0.50
                                 })

Bertmodel = compile_model(Bertmodel)

Bertmodel, history_bert = train_model(Bertmodel, train_bert_inputs_and_masks, y_train, val_bert_inputs_and_masks, y_val, batch_size=128, num_epochs=50, class_weight=class_weight_train)

In [ ]:
gpt2model = tl_gpt2_model(param={'Max_length': 100,
                                  'Trainable' : False,
                                  'first_layer' : 128,
                                  'second_layer' : 64,
                                  'dropout' :0.50
                                 })

gpt2model = compile_model(gpt2model)

gpt2model, history_bert = train_model(gpt2model, train_gpt_inputs_and_masks, y_train, val_gpt_inputs_and_masks, y_val, batch_size=128, num_epochs=50, class_weight=class_weight_train)

In [ ]:
#Assemble model

###Evaluation and Compare Model Performance

In [46]:
test_bert_inputs_and_masks['inputs']

<tf.Tensor: shape=(4957, 60), dtype=int32, numpy=
array([[  101,  1045,  1005, ...,     0,     0,     0],
       [  101,  7570,  2229, ...,     0,     0,     0],
       [  101, 19387,  1030, ...,     0,     0,     0],
       ...,
       [  101, 19387,  1030, ...,     0,     0,     0],
       [  101, 19387,  1030, ...,     0,     0,     0],
       [  101,  1030,  2611, ...,     0,     0,     0]], dtype=int32)>

In [44]:
test = {'inputs':test_bert_inputs_and_masks['inputs'][0:200], 'masks': test_bert_inputs_and_masks['masks'][0:200]}

In [ ]:
eval_dict = evaluate_model(DistilBertmodel, test, y_test[0:200])
eval_dict

In [ ]:
DistilBertmodel(test_bert_inputs_and_masks, training=False)